In [33]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser 
from pydantic import BaseModel, Field 
from typing import Literal 
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
import os 

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
os.environ["HF_HOME"] = os.getenv("HF_HOME")

load_dotenv()

True

In [6]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation"
)
# llm2 = HuggingFaceEndpoint(
#     repo_id="openai/gpt-oss-20b",
#     task="text-generation"
# )

model = ChatHuggingFace(llm=llm)
# model2 = ChatHuggingFace(llm=llm2)

In [ ]:
parser = StrOutputParser()

prompt = PromptTemplate(
    template="Classify the sentiment of the following feedback as positive or negative: \n {feedback}",
    input_variables=["feedback"]
)
classifier_chain = prompt | model | parser

In [8]:
print(classifier_chain.invoke({"feedback": "this is a terrible smartphone"}))

The sentiment of this feedback is negative.


In [ ]:
class Feedback(BaseModel):
    # Literal['positive', 'negative'] 
    sentiment: Literal['positive', 'negative'] = Field(description="Give the sentiment of the feedback as either 'positive' or 'negative'")

parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [ ]:

template = """Classify the sentiment of the following feedback as positive or negative.

Feedback: {feedback}

Return only valid JSON in this exact format:
{format_instruction}
"""

# print(parser2.get_format_instructions())
prompt = PromptTemplate(
    template=template,
    input_variables=["feedback"],
    partial_variables={
        "format_instruction": parser2.get_format_instructions()
    }
)
classifier_chain = prompt | model | parser2
print(classifier_chain.invoke({"feedback": "I dislike the features of this smartphone"}))

sentiment='negative'


In [41]:
prompt_pos = PromptTemplate(
    template="Write an appropriate feedback to this positive feedback \n {feedback}",
    input_variables={"feedback"}
)
prompt_neg = PromptTemplate(
    template="Write an appropriate feedback to this negative feedback \n {feedback}",
    input_variables={"feedback"}
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt_pos | model | parser),
    (lambda x:x.sentiment == 'negative', prompt_neg | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))


# branch_chain = RunnableBranch(
#     (lambda x: x["sentiment"] == "positive", prompt_pos | model | parser),
#     (lambda x: x["sentiment"] == "negative", prompt_neg | model | parser),
#     RunnableLambda(lambda x: "Could not find sentiment")
# )

# # chain = classifier_chain | to_dict | branch_chain 

# to_dict = RunnableLambda(lambda x: x.dict())

# def enrich_input(x, sentiment_result):
#     return {
#         "feedback": x["feedback"],
#         "sentiment": sentiment_result.sentiment
#     }

# merge_chain = RunnableLambda(
#     lambda x: enrich_input(x, x["classifier_output"])
# )

# # chain = {
# #     "feedback": lambda x: x["feedback"],
# #     "classifier_output": classifier_chain
# # } | merge_chain | branch_chain 

# chain = classifier_chain | merge_chain | branch_chain

# print(chain.invoke({"feedback": "This is a bad phone"}))

Here's an example of a constructive response to negative feedback:

"Thank you for taking the time to share your thoughts on my performance. I appreciate your honesty and value your feedback. I can see that there are areas where I can improve, and I'm committed to working on those specific points. Can you please provide more details on what you think I could do differently in the future? I'm eager to learn and grow from this experience."

This response acknowledges the negative feedback, expresses gratitude for the feedback, and shows a willingness to learn and improve. By asking for specific details, you're also showing that you're interested in understanding the feedback and using it to make positive changes.

Alternatively, you could also respond with:

"I appreciate your feedback and I'm sorry to hear that I didn't meet your expectations. I'll take your comments into consideration and work on making improvements. If there's anything specific you'd like me to focus on, please let me